하이퍼파라미터는 사용자가 직접 설정해야 하는 값임
예를 들어 요리를 한다고 가정할 때, 소금을 얼마나 넣을지, 설탕을 얼마나 넣을지, 몇 도로 몇 분을 가열할지 등을 정하는 것임
각 요리에 맞는 조리법이 있듯, 모델에도 최적 하이퍼파라미터를 전달해야 성능이 좋음
모델이 좋은 성능을 내려면 어떤 하이퍼파라미터가 어떤 값을 가지면 좋을지 찾는 작업이 하이퍼파라미터 최적화임
대표적인 최적화 방법에는 그리드서치, 랜덤서치, 베이지안 최적화 이렇게 세 가지가 있음

# 그리드 서치

그리드 서치(Grid Search)는 가장 기본적인 하이퍼파라미터 최적화 기법으로, 주어진 하이퍼파라미터를 모두 순회하며 가장 좋은 성능을 내는 값을 찾음
완전탐색이므로 시간이 매우 오래 걸림

# 랜덤 서치

랜덤 서치(Random Search)는 하이퍼파라미터를 무작위로 탐색하여 가장 좋은 성능을 내는 값을 찾는 기법임
무작위라는 한계 때문에 그리드서치나 베이지안 최적화에 비해 사용 빈도가 떨어짐

랜덤 서치는 사이킷런이 제공하는 RandomizedSearchCV() 메소드로 수행이 가능함
그리드서치와 사용법이 비슷함

# 베이지안 최적화

베이지안 최적화(Bayesian Optimization)란 사전 정보를 바탕으로 최적 하이퍼파라미터 값을 확률적으로 추정하며 탐색하는 기법임
그리드서치나 랜덤서치보다 최적 하이퍼파라미터를 더 빠르고 효율적으로 찾아줌
코드도 직관적이기 때문에 사용하기 편리함

bayes_opt라는 패키지로 베이지안 최적화를 구현할 수 있음
bayes_opt 패키지 기준으로, 다음과 같음

1. 하이퍼파라미터 탐색 범위 설정: 최적 값을 찾고 싶은 하이퍼파라미터의 범위를 설정함
2. 평가지표 계산 함수(성능 평가 함수) 정의: 탐색하려는 파이퍼파라미터를 인수로 받아 평가지표 값을 계산해준느 함수를 정의함
3. BayesianOptimization 객체 생성: bayes_opt 패키지의 BayesianOptimization 객체를 생성함. 객체 생성 시 '평가지표 계산 함수'와 '하이퍼파라미터 탐색 버위'를 입력받음
4. 베이지안 최적화 수행: 3에서 생성한 bayesianOptimization 객체의 maximize() 메소드를 호출함. 하이퍼파라미터 범위 내 값을 평가지표 계산 함수에 전달하면서 평가지표값을 구함. 평가지표 값이 가장 좋았을 때의 하이퍼파라미터 값을 최적 하이퍼파라미터로 간주함

In [1]:
# 하이퍼파라미터 범위(딕셔너리 형태)
param_bounds = {'x': (-5, 5),
                'y': (-4, 4)}

1) 하이퍼파라미터 탐색 범위 설정

이번 예제에서 탐색할 하이퍼파라미터는 x와 y이고, 탐색 범위는 딕셔너리 형태로 지정함
딕셔너리의 키에 하이퍼파라미터 이름을, 값에 하이퍼파라미터 범위(튜플 형태)를 지정함

In [2]:
def eval_function(x, y):
    return -pow(x, 2) - pow(y - 2, 2) + 10

2) 평가지표 계산 함수 정의

베이지안 최적화는 평가지표 계산 함수로 구한 평가점술르 최대화하는 방향으로 하이퍼파라미터를 탐색함
평가점수가 가장 큰 값일 때의 하이퍼파라미터를 최적 하이퍼파라미터로 간주함
물론 실제로 최적값은 아닐 수 있음(최적일 >>가능성<<이 높은 거임)

In [4]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,
                                 pbounds=param_bounds,
                                 random_state=0)

3) 베이지안 최적화 객체 생성

BayesianOptimization()으로 베이지안 최적화 객체를 생성함
중요한 생성 파라미터로는 f와 pbounds가 있음
f에 '최대화하려는 평가지표 계산 함수'를 전달하고, pbounds에 하이퍼파라미터 범위를 전달함
더불어 random_state를 설정해 시드값을 고정하면 다음번에 실행할 때도 동일한 결과를 얻을 수 있음

In [5]:
# 베이지안 최적화 수행
optimizer.maximize(init_points=2, n_iter=10)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | 9.684     | 0.4881    | 1.722     |
| 2         | 6.251     | 1.028     | 0.3591    |
| 3         | -29.64    | -4.909    | -1.942    |
| 4         | -17.32    | 4.829     | 4.0       |
| 5         | 2.015     | -1.996    | 4.0       |
| 6         | -51.0     | 5.0       | -4.0      |
| 7         | -19.0     | -5.0      | 4.0       |
| 8         | 5.116     | 0.94      | 4.0       |
| 9         | 6.346     | -1.131    | 0.4588    |
| 10        | 9.227     | -0.7621   | 2.439     |
| 11        | 9.236     | -0.1571   | 1.14      |
| 12        | 9.167     | 0.002637  | 2.913     |


4) 최적화 수행

최적화는 간단히 maximiza() 메소드로 수행할 수 있음
이 메소드는 여러 파라미터를 받는데, init_points와 n_iter가 제일 중요함

- init_points: 랜덤 탐색을 수행할 스텝 횟수. 랜덤 탐색은 탐색 공간을 다양화함으로써 최적화에 도움을 줄 수 있음
- n_iter: 베이지안 최적화를 수행할 스텝 횟수. 스텝 횟수가 많을수록 최적 값을 찾을 가능성이 높음

In [6]:
# 평가점수가 최대일 때 타깃, x, y값 출력
optimizer.max

{'target': 9.684170249766272,
 'params': {'x': 0.48813503927324753, 'y': 1.7215149309793558}}

총 스텝 횟수(12)는 init_points(2)와 n_iter(10)을 합친 횟수임
중간중간 결과가 빨갛게 출력되는 이유는 평가 함수 점수가 기존 최댓값을 갱신했다는 뜻임
따라서 빨간 결과 중 가장 마지막 스템의 평가점수가 전체에서 최대가 되는 값임
이 결과에서는 11번째 스템으로 베이지안 최적화로 찾은 최적 하이퍼파라미터는 x=0.488, y=1.721임

이 값은 베이지안 최적화 객체의 max에 저장되어 있음